In [1]:
import requests
import json
import time
import re
import pandas as pd

In [7]:
sponsor_party_pattern = re.compile("Sponsor:.*?\[(D|R)-[A-Z][A-Z]\d*?\]")
dem_pattern = re.compile("(\d+?)D")
gop_pattern = re.compile("(\d+?)R")

cosponsor_hist = []
i = 1
URL = "https://www.govtrack.us/congress/bills/browse"
params = {
    "bill_type[]": [2, 4, 8, 6],
    "sort": "-introduced_date",
    "page": 0,
    "faceting": "false",
    "allow_redirect": "true",
    "do_search": 1,
}
i = 1
for i in range(1, 5000):
    params["page"] = i
    i += 1
    if i % 100 == 0:
        print(i)
    web_data = json.loads(requests.get(URL, params=params).text)
    if not web_data.get("results", False):
        break
    for result in web_data["results"]:
        bill_cosponsors = (
            pd.read_html(result)[0][[0, 2]]
            .rename(columns={0: "date", 2: "cosponsor_parties"})
            .to_dict(orient="records")
        )
        match = sponsor_party_pattern.search(result)
        if match is None:
            continue
        bill_cosponsors[0]["sponsor_party"] = match.group(1)
        cosponsor_hist += bill_cosponsors
    time.sleep(0.2)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [8]:
df = pd.DataFrame(cosponsor_hist)

In [9]:
df["dem_cosponsors"] = df["cosponsor_parties"].str.extract(dem_pattern).fillna(0)
df["gop_cosponsors"] = df["cosponsor_parties"].str.extract(gop_pattern).fillna(0)
df["date"] = pd.to_datetime(df["date"].str.replace("Introduced", ""))
df = df.set_index("date").sort_index().drop(columns="cosponsor_parties")

In [10]:
df

,sponsor_party,dem_cosponsors,gop_cosponsors
date,,,
1974-02-06,R,11,9
1974-02-06,R,0,0
1974-02-06,D,0,1
1974-02-06,D,0,1
1974-02-06,D,1,0
...,...,...,...
2021-05-25,D,1,0
2021-05-25,D,0,0
2021-05-25,D,0,2
